In [ ]:
import numpy as np
import numpy.random as npr
import matplotlib.cm as cm
import matplotlib.pyplot as plt


In [ ]:
# functions
def noise(shape):
    return 1/np.sqrt(2) * (npr.randn(shape) + 1.0j * npr.randn(shape))

def to_db(x):
    return 20 * np.log10(np.abs(x))

def chirp(
    time_support,
    amp,
    fc,
    chirp_rate,
    t_delay
):
    return amp*np.exp(1.0j*2*np.pi*(fc*(time_support - t_delay) + chirp_rate*(time_support-t_delay)**2))

In [ ]:
# Define signal
fc = 0
bandwidth = 10  # Hz
nyq_fact = 20
fs = nyq_fact * 2 * bandwidth
ts = 1/fs
duration = 25
chirp_rate = bandwidth/duration

time_support = np.arange(0, duration, step=ts)
n_pt = time_support.size

# t_delays = [1.7,4]
t_delays = [0, 1.1, duration/4]
amp = 2
x = np.zeros((len(t_delays), n_pt), dtype=np.complex128)

for ii, t_delay in enumerate(t_delays):
    
    x[ii, :] = chirp(
        time_support=time_support,
        amp=amp,
        fc=fc,
        chirp_rate=chirp_rate,
        t_delay=t_delay
    )

n = noise(n_pt)

sig = x + n


In [ ]:
# View signal
_, axs = plt.subplots(
    nrows=1,
    ncols=2,
    clear=True,
    num=0
)

for ii in range(len(t_delays)):
    axs[0].plot(time_support, np.real(x[ii,:]))
    axs[1].plot(time_support, np.real(sig[ii,:]))
plt.show()


In [ ]:
# Matched filter
sig_freq = np.fft.fft(sig, axis=-1)
conj_reverse = np.fft.fft(np.conj(sig), axis=-1)
matched_freq = sig_freq * conj_reverse

matched = np.fft.ifft(matched_freq, axis=-1) / np.sqrt(n_pt)  # normalize gain


In [ ]:
_, axs = plt.subplots(
    nrows=1,
    ncols=1,
    clear=True,
    num=1,
    figsize=(8,8)
)

sig_db = to_db(matched)

colors = cm.get_cmap('jet', len(t_delays))
for ii, t_delay in enumerate(t_delays):
    axs.plot(time_support/2, sig_db[ii, :], color=colors(ii))
    axs.plot([t_delay, t_delay], plt.gca().get_ylim(), '--', color=colors(ii))
plt.ylim(sig_db.max()-70, sig_db.max()+3)
plt.show()
